<a href="https://colab.research.google.com/github/jamesm2002/tone_based_counterspeech_models/blob/main/T5_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Counterspeech model T5 Training

In [ ]:
!pip install transformers datasets torch

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, get_scheduler
from datasets import load_dataset
from torch.optim import AdamW
from transformers import TrainerCallback

# load T5 tokenizer and model
MODEL_NAME = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

# load dataset and split into train/validation (90% train, 10% validation)
dataset = load_dataset("csv", data_files={"train": "combined_CONAN_with_tone.csv"})
dataset = dataset["train"].train_test_split(test_size=0.1)

# preprocessing function
def preprocess_function(examples):
    tone_instructions = {
        "Inquisitive": "Ask a thoughtful question about the statement:",
        "Confrontational": "Strongly refute the statement with counter-evidence:",
        "Empathetic": "Acknowledge the concern but provide a hopeful perspective:",
        "Conversational": "Respond in a casual and friendly way:"
    }

    inputs = [
        f"{tone_instructions[tone]} {text}"
        for tone, text in zip(examples["TONE"], examples["HATE_SPEECH"])
    ]
    targets = examples["COUNTER_NARRATIVE"]

    model_inputs = tokenizer(inputs, max_length=150, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# tokenize dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# training arguments
training_args = TrainingArguments(
    output_dir="./t5-counterspeech",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=100,
    learning_rate=1.5e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.02,
    weight_decay=0.02,
    num_train_epochs=9,
    save_total_limit=2,
    push_to_hub=False,
    fp16=False,
    max_grad_norm=1.0,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
)

# optimiser, warm-up and cosine decay

optimizer = AdamW(model.parameters(), lr=1.5e-5)

num_training_steps = len(tokenized_datasets["train"]) // training_args.per_device_train_batch_size * training_args.num_train_epochs
num_warmup_steps = int(0.01 * num_training_steps)

optimizer = AdamW(model.parameters(), lr=1.5e-5)

lr_scheduler = get_scheduler(
    name="cosine",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
)


# learning rate logger
class LrLoggerCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            log_message = f"Step {state.global_step}: "
            if "learning_rate" in logs:
                log_message += f"Learning Rate = {logs['learning_rate']:.8f} "
            if "loss" in logs:
                log_message += f"Training Loss = {logs['loss']:.4f} "
            if "eval_loss" in logs:
                log_message += f"Validation Loss = {logs['eval_loss']:.4f}"
            print(log_message)

# trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    optimizers=(optimizer, lr_scheduler),
    callbacks=[LrLoggerCallback()]
)

# train model
trainer.train()

# save the fine-tuned model
model.save_pretrained("./t5-counterspeech-model")
tokenizer.save_pretrained("./t5-counterspeech-model")

print("Training complete! Model saved to ./t5-counterspeech-model")




In [ ]:
# generate example responses

def generate_response(hate_speech, tone):
    input_text = f"<{tone}> Generate an {tone.lower()} counterspeech response to the following: {hate_speech}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

    output_ids = model.generate(
        input_ids,
        max_length=150,
        num_beams=5,
        no_repeat_ngram_size=3,
        temperature=1.7,
        top_p=0.85,
        repetition_penalty=1.5,
        early_stopping=True
    )

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    response = response.replace("<extra_id_0>", "").strip()

    return response


# Example test
print(generate_response("Ask a thoughtful question about the statement: Gays are destroying Europe.", "Inquisitive"))
print(generate_response("Strongly refute the statement with counter-evidence:", "Confrontational"))
print(generate_response("Muslims are destroying Europe.", "Empathetic"))
print(generate_response("Muslims are destroying Europe.", "Conversational"))



In [ ]:
model_path = "./t5-counterspeech-tone-model"
save_path = "/content/drive/My Drive/t5-counterspeech-tone-model"

# save the model & tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to {save_path}")